In [1]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as BraveService
from webdriver_manager.core.utils import ChromeType
import json
import time
from tqdm.notebook import tqdm 

<h4> Start cells here if partial FIPS already available in {state}_fips_scraped file

In [2]:
fips_check_scraped = pd.read_csv("../../../data/state_data/geo/geo_fips/23/NJ_fips_scraped.csv") 
fips_check_scraped  = fips_check_scraped.loc[:, ~fips_check_scraped.columns.str.contains('^Unnamed')] 
fips_check_scraped.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,1933087308,NJ,"2050 STATE ROUTE 27, NORTH BRUNSWICK, NJ",40.457578,-74.510531,3.402301e+14
1,2697537303,NJ,"331 NEWMAN SPRINGS RD BLDG 3 STE 350 2ND FLR, ...",40.334716,-74.095448,3.402580e+14
2,3055997206,NJ,"2535 John F. Kennedy Blvd, Jersey City, NJ",40.723953,-74.074761,3.401700e+14
3,3162367309,NJ,"78 OKNER PARKWAY, LIVINGSTON, NJ",40.803235,-74.352090,3.401302e+14
4,3622487203,NJ,"1 Radar Way, EATONTOWN, NJ",40.296082,-74.084398,3.402581e+14


In [3]:
#Latitude, Longitude, not included
fips_check_scraped_notna = fips_check_scraped[fips_check_scraped["Lat"].notna()] # check to make sure no nulls in coords
fips_check_scraped_notna.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,1933087308,NJ,"2050 STATE ROUTE 27, NORTH BRUNSWICK, NJ",40.457578,-74.510531,3.402301e+14
1,2697537303,NJ,"331 NEWMAN SPRINGS RD BLDG 3 STE 350 2ND FLR, ...",40.334716,-74.095448,3.402580e+14
2,3055997206,NJ,"2535 John F. Kennedy Blvd, Jersey City, NJ",40.723953,-74.074761,3.401700e+14
3,3162367309,NJ,"78 OKNER PARKWAY, LIVINGSTON, NJ",40.803235,-74.352090,3.401302e+14
4,3622487203,NJ,"1 Radar Way, EATONTOWN, NJ",40.296082,-74.084398,3.402581e+14


In [4]:
#FIPS not included, of Lat, Long available
fips_check_scraped_null = fips_check_scraped_notna[fips_check_scraped_notna["FIPS_z"].isnull()] # check to make sure no nulls in coords
fips_check_scraped_null #28093

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
867,3144267702,NJ,"101 COLLEGE ROAD EAST 2ND FLOOR, PRINCETON, NJ",40.358640,-74.601988,NaN
990,7602717103,NJ,"451 Frelinghuysen Avenue, NEWARK, NJ",40.705687,-74.194178,NaN
1172,8274828407,NJ,"27 RIDGE RD, WHITEHOUSE STATION, NJ",40.605363,-74.735444,NaN
1431,3601787701,NJ,"5140 US Hwy 9, howell, NJ",34.251379,-85.453067,NaN
2623,4714287106,NJ,"4 BELL DRIVE, RIDGEFIELD, NJ",40.837074,-74.012039,NaN
3537,3317248702,NJ,"601 Bangs Ave Suite 402, ASBURY PARK, NJ",40.217103,-74.010180,NaN
3554,4945367310,NJ,"2300 US Highway 1 North, ENGLISHTOWN, NJ",38.930717,-76.973348,NaN
3658,8609857205,NJ,"2 Riverview Dr, SOMERSET, NJ",40.547475,-74.554659,NaN
4784,4879448405,NJ,"91 MILLBROOK CIR, NORWOOD, NJ",40.977720,-73.934398,NaN
5184,7363288408,NJ,"10 Park Pl Bldg 1A Ste 504, Butler, NJ",40.747720,-73.981308,NaN


<h4>FIPS Script

In [5]:
#if FIPS_z file 
lat = fips_check_scraped_null["Lat"].astype(float).astype(str).to_list()
long = fips_check_scraped_null["Long"].astype(float).astype(str).to_list()

In [6]:
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)
option = webdriver.ChromeOptions()
option.binary_location = "/Applications/Brave Browser.app/Contents/MacOS/Brave Browser"
driver = Browser('chrome', options=option, headless=False)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('geoCoordX').clear()
        driver.find_by_id('geoCoordY').clear()
        driver.find_by_id('geoCoordX').fill(long[i])
        driver.find_by_id('geoCoordY').fill(lat[i])
        driver.find_by_id('geoCoordsSubmit').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='geoCoordsJsonOutput')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('34')] #NJ
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                #print(f"{i},{lat[i]},{long[i]} Not Found")
                not_found.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1

  0%|          | 0/49 [00:00<?, ?it/s]

coordinates wtih more than one return:
385,37.514070000000004,-77.35472 is ['514070000000004', '510872015041003'] 
1178,37.516454700000004,-77.8100575 is ['516454700000004', '511455001022014'] 
2660,37.519462299999994,-77.340384 is ['519462299999994', '510872014061012']
second option reference FIPS code, first option references LatCoord, update fields in DataFrame

https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

In [7]:
geo_id_list =geo_id 

Length Check

In [8]:
print(len(geo_id_list))

49


Matching FIPS to DataFrame

In [9]:
fips_fill = fips_check_scraped_null

In [10]:
fips_fill["FIPS_z"] = geo_id_list
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_9734/597263667.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill["FIPS_z"] = geo_id_list


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
867,3144267702,NJ,"101 COLLEGE ROAD EAST 2ND FLOOR, PRINCETON, NJ",40.358640,-74.601988,[340230086012012]
990,7602717103,NJ,"451 Frelinghuysen Avenue, NEWARK, NJ",40.705687,-74.194178,[340130048021017]
1172,8274828407,NJ,"27 RIDGE RD, WHITEHOUSE STATION, NJ",40.605363,-74.735444,[340190111002025]
1431,3601787701,NJ,"5140 US Hwy 9, howell, NJ",34.251379,-85.453067,[]
2623,4714287106,NJ,"4 BELL DRIVE, RIDGEFIELD, NJ",40.837074,-74.012039,[340030452003006]
3537,3317248702,NJ,"601 Bangs Ave Suite 402, ASBURY PARK, NJ",40.217103,-74.010180,[340258070052023]
3554,4945367310,NJ,"2300 US Highway 1 North, ENGLISHTOWN, NJ",38.930717,-76.973348,[]
3658,8609857205,NJ,"2 Riverview Dr, SOMERSET, NJ",40.547475,-74.554659,[340350535012029]
4784,4879448405,NJ,"91 MILLBROOK CIR, NORWOOD, NJ",40.977720,-73.934398,[340030023002014]
5184,7363288408,NJ,"10 Park Pl Bldg 1A Ste 504, Butler, NJ",40.747720,-73.981308,[]


Remove brackets

In [11]:
fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_9734/1219885402.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
867,3144267702,NJ,"101 COLLEGE ROAD EAST 2ND FLOOR, PRINCETON, NJ",40.358640,-74.601988,340230086012012
990,7602717103,NJ,"451 Frelinghuysen Avenue, NEWARK, NJ",40.705687,-74.194178,340130048021017
1172,8274828407,NJ,"27 RIDGE RD, WHITEHOUSE STATION, NJ",40.605363,-74.735444,340190111002025
1431,3601787701,NJ,"5140 US Hwy 9, howell, NJ",34.251379,-85.453067,NaN
2623,4714287106,NJ,"4 BELL DRIVE, RIDGEFIELD, NJ",40.837074,-74.012039,340030452003006
3537,3317248702,NJ,"601 Bangs Ave Suite 402, ASBURY PARK, NJ",40.217103,-74.010180,340258070052023
3554,4945367310,NJ,"2300 US Highway 1 North, ENGLISHTOWN, NJ",38.930717,-76.973348,NaN
3658,8609857205,NJ,"2 Riverview Dr, SOMERSET, NJ",40.547475,-74.554659,340350535012029
4784,4879448405,NJ,"91 MILLBROOK CIR, NORWOOD, NJ",40.977720,-73.934398,340030023002014
5184,7363288408,NJ,"10 Park Pl Bldg 1A Ste 504, Butler, NJ",40.747720,-73.981308,NaN


In [12]:
fips_fill.loc[fips_fill["FIPS_z"].isnull()] #check #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
1431,3601787701,NJ,"5140 US Hwy 9, howell, NJ",34.251379,-85.453067,NaN
3554,4945367310,NJ,"2300 US Highway 1 North, ENGLISHTOWN, NJ",38.930717,-76.973348,NaN
5184,7363288408,NJ,"10 Park Pl Bldg 1A Ste 504, Butler, NJ",40.747720,-73.981308,NaN
5761,1603408301,NJ,"450 US Highway 1 and 9, Newark, NJ",27.375662,-80.326119,NaN
5771,8351957104,NJ,"761 RT 9, BAYVILLE, NJ",43.818420,-73.772538,NaN
8260,3438857305,NJ,"1460 US HWY 9 N SUITE 203, WOODBRIDGE, NJ",34.251379,-85.453067,NaN
8760,7836737003,NJ,"450 US HIGHWAY 1 AND 9, NEWARK, NJ",27.375662,-80.326119,NaN
12062,2617848702,NJ,"749 E. Goepp Street, Bethlehem, NJ",40.626382,-75.361486,NaN
15646,1071227210,NJ,"1601 Market Street 2300, Philadelphia, NJ",39.953097,-75.167352,NaN
17206,2984238306,NJ,"2720 US Highway 1, Lawrence, NJ",44.499859,-68.146898,NaN


Parse for Merge

In [13]:
cols= ["LoanNumber", "FIPS_z"]
fips_fill_parsed = fips_fill[cols]
fips_fill_parsed

,LoanNumber,FIPS_z
867,3144267702,340230086012012
990,7602717103,340130048021017
1172,8274828407,340190111002025
1431,3601787701,NaN
2623,4714287106,340030452003006
3537,3317248702,340258070052023
3554,4945367310,NaN
3658,8609857205,340350535012029
4784,4879448405,340030023002014
5184,7363288408,NaN


Read in base data

In [14]:

#if from FIPS file
fips_check = pd.read_csv("../../../data/state_data/geo/geo_fips/23/NJ_fips_scraped.csv") 
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0"], axis =1) 
fips_check.head()

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,558332,1933087308,NJ,"2050 STATE ROUTE 27, NORTH BRUNSWICK, NJ",40.457578,-74.510531,3.402301e+14
1,558333,2697537303,NJ,"331 NEWMAN SPRINGS RD BLDG 3 STE 350 2ND FLR, ...",40.334716,-74.095448,3.402580e+14
2,558334,3055997206,NJ,"2535 John F. Kennedy Blvd, Jersey City, NJ",40.723953,-74.074761,3.401700e+14
3,558335,3162367309,NJ,"78 OKNER PARKWAY, LIVINGSTON, NJ",40.803235,-74.352090,3.401302e+14
4,558336,3622487203,NJ,"1 Radar Way, EATONTOWN, NJ",40.296082,-74.084398,3.402581e+14


Merge with MainFrame

In [15]:
fips_merge = fips_check.merge(fips_fill_parsed, on="LoanNumber", how="left")
fips_merge

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y
0,558332,1933087308,NJ,"2050 STATE ROUTE 27, NORTH BRUNSWICK, NJ",40.457578,-74.510531,3.402301e+14,NaN
1,558333,2697537303,NJ,"331 NEWMAN SPRINGS RD BLDG 3 STE 350 2ND FLR, ...",40.334716,-74.095448,3.402580e+14,NaN
2,558334,3055997206,NJ,"2535 John F. Kennedy Blvd, Jersey City, NJ",40.723953,-74.074761,3.401700e+14,NaN
3,558335,3162367309,NJ,"78 OKNER PARKWAY, LIVINGSTON, NJ",40.803235,-74.352090,3.401302e+14,NaN
4,558336,3622487203,NJ,"1 Radar Way, EATONTOWN, NJ",40.296082,-74.084398,3.402581e+14,NaN
...,...,...,...,...,...,...,...,...
32510,590842,9754798404,NJ,"219 S Main St, Barnegat, NJ",39.745114,-74.231276,3.402973e+14,NaN
32511,590843,9776128806,NJ,"303 Doremus Ave 303 Doremus Ave, Newark, NJ",40.723252,-74.126015,3.401301e+14,NaN
32512,590844,9818008107,NJ,"830 GROVE ST, ELIZABETH, NJ",40.655283,-74.218870,3.403903e+14,NaN
32513,590845,9876707706,NJ,"1256 Indian Head Rd, Toms River, NJ",39.991767,-74.215181,3.402972e+14,NaN


<b>Fill FIPS Columns - if FIPS file

In [16]:
fips_merge["FIPS_z"] = fips_merge["FIPS_z_y"].fillna(fips_merge["FIPS_z_x"])
fips_merge.head()

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y,FIPS_z
0,558332,1933087308,NJ,"2050 STATE ROUTE 27, NORTH BRUNSWICK, NJ",40.457578,-74.510531,3.402301e+14,NaN,340230062051009.0
1,558333,2697537303,NJ,"331 NEWMAN SPRINGS RD BLDG 3 STE 350 2ND FLR, ...",40.334716,-74.095448,3.402580e+14,NaN,340258013005019.0
2,558334,3055997206,NJ,"2535 John F. Kennedy Blvd, Jersey City, NJ",40.723953,-74.074761,3.401700e+14,NaN,340170041041000.0
3,558335,3162367309,NJ,"78 OKNER PARKWAY, LIVINGSTON, NJ",40.803235,-74.352090,3.401302e+14,NaN,340130204004013.0
4,558336,3622487203,NJ,"1 Radar Way, EATONTOWN, NJ",40.296082,-74.084398,3.402581e+14,NaN,340258126004030.0


Drop Extra FIPS Columns

In [17]:
fips_merge_drop = fips_merge.drop(["FIPS_z_x", "FIPS_z_y"], axis=1)
fips_merge_drop = fips_merge_drop.drop(["Unnamed: 0"], axis=1)
fips_merge_drop

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,1933087308,NJ,"2050 STATE ROUTE 27, NORTH BRUNSWICK, NJ",40.457578,-74.510531,340230062051009.0
1,2697537303,NJ,"331 NEWMAN SPRINGS RD BLDG 3 STE 350 2ND FLR, ...",40.334716,-74.095448,340258013005019.0
2,3055997206,NJ,"2535 John F. Kennedy Blvd, Jersey City, NJ",40.723953,-74.074761,340170041041000.0
3,3162367309,NJ,"78 OKNER PARKWAY, LIVINGSTON, NJ",40.803235,-74.352090,340130204004013.0
4,3622487203,NJ,"1 Radar Way, EATONTOWN, NJ",40.296082,-74.084398,340258126004030.0
...,...,...,...,...,...,...
32510,9754798404,NJ,"219 S Main St, Barnegat, NJ",39.745114,-74.231276,340297340031014.0
32511,9776128806,NJ,"303 Doremus Ave 303 Doremus Ave, Newark, NJ",40.723252,-74.126015,340130074001003.0
32512,9818008107,NJ,"830 GROVE ST, ELIZABETH, NJ",40.655283,-74.218870,340390307031001.0
32513,9876707706,NJ,"1256 Indian Head Rd, Toms River, NJ",39.991767,-74.215181,340297230002002.0


In [18]:
fips_merge_drop.loc[fips_merge_drop["FIPS_z"].isnull()] #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
1431,3601787701,NJ,"5140 US Hwy 9, howell, NJ",34.251379,-85.453067,NaN
3554,4945367310,NJ,"2300 US Highway 1 North, ENGLISHTOWN, NJ",38.930717,-76.973348,NaN
5184,7363288408,NJ,"10 Park Pl Bldg 1A Ste 504, Butler, NJ",40.747720,-73.981308,NaN
5761,1603408301,NJ,"450 US Highway 1 and 9, Newark, NJ",27.375662,-80.326119,NaN
5771,8351957104,NJ,"761 RT 9, BAYVILLE, NJ",43.818420,-73.772538,NaN
8260,3438857305,NJ,"1460 US HWY 9 N SUITE 203, WOODBRIDGE, NJ",34.251379,-85.453067,NaN
8760,7836737003,NJ,"450 US HIGHWAY 1 AND 9, NEWARK, NJ",27.375662,-80.326119,NaN
12062,2617848702,NJ,"749 E. Goepp Street, Bethlehem, NJ",40.626382,-75.361486,NaN
15646,1071227210,NJ,"1601 Market Street 2300, Philadelphia, NJ",39.953097,-75.167352,NaN
17206,2984238306,NJ,"2720 US Highway 1, Lawrence, NJ",44.499859,-68.146898,NaN


Convert to File

In [19]:
fips_NJ = fips_merge_drop

In [20]:
fips_NJ.to_csv("../../../data/state_data/geo/geo_fips/23/NJ_fips_scraped.csv") 